<a href="https://colab.research.google.com/github/kattens/ABUS/blob/main/Junk_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install PubChemPy

In [ ]:
import csv
import pandas as pd
import pubchempy as pcp
import os
import requests

In [ ]:
#THE PRACTICE CODE FOR SEEING IF THE REQUEST RETURNS THE PAGE WE WANT!
'''
import requests
import webbrowser
from IPython.display import IFrame

def fetch_bioassay_results(cid):
    query = f'https://pubchem.ncbi.nlm.nih.gov/compound/{cid}#section=BioAssay-Results&fullscreen=true'

    try:
        # Try opening in a new tab (might not work in all environments)
        webbrowser.open_new_tab(query)
    except Exception as e:
        print(f"Error opening in new tab: {e}")
        # Fallback: Open in default browser (new window or tab)
        webbrowser.open(query)

    # Embed the webpage in the notebook using IFrame
    display(IFrame(query, width=800, height=600))

    # Make the request with 'requests' (optional)
    response = requests.get(query)
    if response.status_code == 200:
        print(f"Successfully accessed the website for CID: {cid}")
    else:
        print(f"Failed to access the website for CID: {cid}, Status code: {response.status_code}")

# Example usage
fetch_bioassay_results(5330175)
'''

In [ ]:
#code for using the PUG API in pubchem:

'''
import requests

def fetch_and_download_csv(cid):
    # Update the CSV URL you identified
    csv_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/bioassay/csv"

    # Fetch the CSV file
    try:
        response = requests.get(csv_url)
        if response.status_code == 200:
            # Save the CSV file locally
            file_name = f"bioassay_results_{cid}.csv"
            with open(file_name, "wb") as file:
                file.write(response.content)
            print(f"CSV file downloaded successfully as {file_name}")
        else:
            print(f"Failed to fetch the CSV file. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
fetch_and_download_csv(5330175)

'''

In [ ]:
#give example from the bioessay_dict with the first entry
#order -> view object -> list of tuples -> every tuple is (str, list of str)
tuple_61 = list(bioessay_dict.items())[61]
print(tuple_61[0])
print(tuple_61[1])
#check how many elements are there in the value which is a list
print(len(list(bioessay_dict.values())[61]))
#also drop the values that are similar only keep the unique ones in the list
print(len(set(list(bioessay_dict.values())[61])))
print(list(set(list(bioessay_dict.values())[61])))

In [ ]:
#When you iterate over a dictionary directly (e.g., for key in bioessay_dict), Python defaults to iterating over the dictionary's keys.
#The __iter__ method of a dictionary returns an iterator over the keys. (built in defualt)

# Create a new dictionary with updated keys
updated_bioessay_dict = {}
for key in bioessay_dict:
    new_key = key.replace('.csv', '')  # Remove .csv from the key
    updated_bioessay_dict[new_key] = bioessay_dict[key]  # Assign the value to the new key

# Replace the old dictionary with the updated one
bioessay_dict = updated_bioessay_dict

print(bioessay_dict.keys())

# **Our Goal**

Now we want a dataset structure that:

    Stores this triplet clearly

    Allows you to:

        Run sequence alignments between target and malaria proteins

        Analyze which drugs may bind to malaria proteins

        Potentially train or evaluate a model later

✅ Recommended Dataset Format (JSONL or DataFrame)

🧬 Each row represents a single target–malaria protein match

{
  "pubchem_id": "4735",
  "target_chain_id": "1RKW_A",
  "target_sequence": "MVLSPADKTN...",
  "malaria_match_id": "3D7A_A",
  "malaria_sequence": "MVLSPADKTV...",
  "percent_identity": 85.2,
  "alignment_length": 120,
  "evalue": 1e-50,
  "bitscore": 240.0
}

In [ ]:
#Create the Ultimate Dataset:
import pandas as pd
import json
import os
json_path = '/content/drive/MyDrive/Drug Repurposing Project/PubChem_PDB_Results'
#Make the main dataset
columns = ['pubchem_id', 'target_chain_id', 'target_sequence', 'malaria_match_id', 'malaria_sequence', 'percent_identity', 'alignment_length', 'evalue', 'bitscore']
df = pd.DataFrame(columns=columns)

✅ What it does:

    Each folder = a PubChem ID

    Each file = a target name (e.g., a protein chain)

    Each file contains a list of BLAST hit dictionaries

    You want one row per hit, with all relevant info

In [ ]:
# Traverse each PubChem ID folder
for pubchem_id in os.listdir(json_path):
    folder_path = os.path.join(json_path, pubchem_id)
    if not os.path.isdir(folder_path):
        continue

    for file in os.listdir(folder_path):
        if not file.endswith('.json'):
            continue
        target_chain_id = file.replace('.json', '')
        file_path = os.path.join(folder_path, file)

        with open(file_path, 'r') as f:
            try:
                data = json.load(f)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
                continue

        # Skip if message says "No result found."
        if isinstance(data, list) and len(data) > 0 and "message" in data[0]:
            if data[0]["message"] == "No result found.":
                continue

        for hit in data:
            row = {
                'pubchem_id': pubchem_id,
                'target_chain_id': target_chain_id,
                'malaria_match_id': hit.get("subject_id"),
                'percent_identity': hit.get("percent_identity"),
                'alignment_length': hit.get("alignment_length"),
                'evalue': hit.get("evalue"),
                'bitscore': hit.get("bitscore")
            }
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

# Preview or save
print(df.head())
# df.to_csv("malaria_alignment_dataset.csv", index=False)
df.to_csv("final_dataset.csv", index=False)

#🧬 Core Goal of the Pipeline

The pipeline aims to identify potential repurposing opportunities for existing drugs by leveraging known drug-target interactions and structural homology with malaria proteins. Specifically:

  - For each drug in the dataset:
  - Retrieve the known protein targets (human or other) that the drug interacts with.

  - For each target:

      - Download the corresponding PDB structure.

      - Extract and isolate individual protein chains.

   - For each chain:
   - Perform BLASTp search against the PDB database restricted to Plasmodium falciparum (taxID: 5833) to identify structurally similar malaria proteins.

   - For each matched malaria protein:

      - If a PDB structure is available, download it.

      - If only a UniProt ID exists, retrieve the AlphaFold predicted structure.

Final Objective:
  - Compare the known drug-target interaction site with the structurally similar malaria protein site.
  - If significant structural similarity is observed, infer that the drug may also bind to the malaria protein at the same or similar site → suggesting a potential for drug repurposing.

Summary:

Drug → Known Target → Structure → Similar Malaria Protein → Binding Prediction

In [ ]:
# --- 1. Configuration and Paths ---

#have the paths set, if not existing, make them
BASE_PATH = '/content/drive/MyDrive/Drug Repurposing Project/Interactions_Results'
SAVE_PATH = '/content/drive/MyDrive/Drug Repurposing Project/PubChem_PDB_Results'
PDB_SAVE_PATH = os.path.join(SAVE_PATH, "pdb_matches")

os.makedirs(SAVE_PATH, exist_ok=True)
os.makedirs(PDB_SAVE_PATH, exist_ok=True)

# We are combining all interaction and pathway files along with PubChem IDs and other relevant information, we will also focus on using the target PDB ID for further processing.
Interaction_folder_path = BASE_PATH

# 🔄 Combine all CSV files into one DataFrame
combined_df = pd.DataFrame()

for filename in os.listdir(Interaction_folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(Interaction_folder_path, filename)
        df = pd.read_csv(file_path)
        df['source_file'] = filename  # Optional: keep track of original source
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# ➕ Add pubchem_id by removing '.csv' and converting to integer
combined_df['pubchem_id'] = combined_df['source_file'].str.replace('.csv', '', regex=False).astype(int)

# 💾 Save the combined DataFrame to a new CSV
output_path = os.path.join(Interaction_folder_path, '/content/drive/MyDrive/Drug Repurposing Project/Combined_Interaction_PDB_Targets.csv')
combined_df.to_csv(output_path, index=False)

print(f"✅ Combined CSV saved to: {output_path}")

df = pd.read_csv('/content/drive/MyDrive/Drug Repurposing Project/Combined_Interaction_PDB_Targets.csv')
df.columns

### 🧬 Key Column Descriptions – `combined_targets.csv`

| Column Name     | Description                                                                 |
|------------------|-----------------------------------------------------------------------------|
| `pdbid`          | PDB ID of the protein-ligand structure (e.g., `4I24`)                       |
| `title`          | Title/description of the structure (often includes protein and ligand info)|
| `expmethod`      | Method used to determine the structure (e.g., X-RAY DIFFRACTION)            |
| `resolution`     | Resolution of the structure in Ångströms; lower values = better quality     |
| `lignme`         | Ligand(s) present in the structure (e.g., `CLQ` = chloroquine)              |
| `cids`           | PubChem Compound IDs (CIDs) for ligands in the structure                    |
| `protacxns`      | UniProt accession ID(s) or protein IDs involved in the interaction          |
| `geneids`        | NCBI Gene IDs corresponding to the proteins                                 |
| `pmid`           | PubMed ID of the publication describing the structure                       |
| `dois`           | DOI (Digital Object Identifier) for the structure's publication             |
| `pmcids`         | PubMed Central ID, if available                                              |
| `pclids`         | PubChem Literature IDs                                                       |
| `citations`      | Full text reference or author list                                          |
| `source_file`    | Name of the original CSV file (e.g., `444810.csv`) that this row came from  |


# ✅ Step 1: Download PDB Structures of Drug Targets

We have a list of drugs (PubChem CIDs) and their known protein targets.

Goal:
Find the 3D structure (PDB file) of each protein target (the human protein that interacts with the drug).

How:
For each PubChem CID + Target:

    Search RCSB Protein Data Bank to see if there’s a known structure.

    If found, download the .pdb or .cif file and save it.

This gives you the structures of the human proteins that the drugs are known to interact with.

In [ ]:
# This is just a simple query to get the pubchem_id and pdb_id as one query
def pdb_query(pubchem_id, pdb_id):
    return {
        "query": {
            "type": "group",
            "logical_operator": "and",
            "nodes": [
                {
                    "type": "group",
                    "logical_operator": "and",
                    "nodes": [
                        {
                            "type": "terminal",
                            "service": "text_chem",
                            "parameters": {
                                "attribute": "rcsb_chem_comp_related.resource_accession_code",
                                "operator": "exact_match",
                                "negation": False,
                                "value": pubchem_id  # string, not list
                            }
                        },
                        {
                            "type": "terminal",
                            "service": "text_chem",
                            "parameters": {
                                "attribute": "rcsb_chem_comp_related.resource_name",
                                "operator": "exact_match",
                                "value": "PubChem",
                                "negation": False
                            }
                        }
                    ],
                    "label": "nested-attribute"
                },
                {
                    "type": "terminal",
                    "service": "full_text",
                    "parameters": {
                        "value": pdb_id  # string, not list
                    }
                }
            ]
        },
        "return_type": "entry",
        "request_options": {
            "paginate": {
                "start": 0,
                "rows": 25
            },
            "results_content_type": [
                "experimental"
            ],
            "sort": [
                {
                    "sort_by": "score",
                    "direction": "desc"
                }
            ],
            "scoring_strategy": "combined"
        }
    }



In [ ]:
def download_pdb(query, file_dir):
    # Step 1: Send query to RCSB -> this work the same as the regex url address modified by previous code
    url = "https://search.rcsb.org/rcsbsearch/v2/query"
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, headers=headers, data=json.dumps(query))

    if response.status_code != 200:
        print(f"Query failed: {response.status_code}")
        print(response.text)
        return

    # Step 2: Extract PDB IDs
    result = response.json()
    pdb_ids = [entry["identifier"] for entry in result.get("result_set", [])]
    if not pdb_ids:
        print("No PDB entries found.")
        return

    print(f"Found PDB IDs: {pdb_ids}")

    # Step 3: Download PDB files
    os.makedirs(file_dir, exist_ok=True)
    for pdb_id in pdb_ids:
        pdb_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
        pdb_response = requests.get(pdb_url)
        if pdb_response.status_code == 200:
            with open(os.path.join(file_dir, f"{pdb_id}.pdb"), "w") as f:
                f.write(pdb_response.text)
            print(f"Downloaded: {pdb_id}")
        else:
            print(f"Failed to download: {pdb_id}")


#mechanism to download the pdb file and save them in a folder named after the pubchem id  in our target folder
#target folder -> make folders with pubchem id -> save each related pdb in it
for index, row in df.iterrows():
    pubchem_id = row['pubchem_id']
    pdb_id = row['pdbid']
    query = pdb_query(str(pubchem_id), pdb_id)
    download_pdb(query,f'/content/drive/MyDrive/Drug Repurposing Project/PubChem_PDB_Results/{pubchem_id}')
    print(f"Processed PubChem ID: {pubchem_id}")

# ✅ Step 2: BLAST Against Malaria Proteins

Now you have the sequences of drug target proteins.

Goal:
Find if these human targets are similar to any malaria proteins (Taxonomy ID 5833).

How:
You run BLASTp (protein-protein alignment):

    For each chain FASTA sequence

    Against a filtered BLAST database of only malaria proteins

    Save the results (scores, % identity, alignment, PDB ID of malaria proteins)

If you find similar malaria proteins → there’s a chance the drug may bind to them.

#🎯 Why Chain Separation is Necessary

A PDB file may contain:

    One protein chain → or

    Multiple chains (complexes, homo-oligomers, hetero-oligomers)

When you want to find if a specific part (a chain) of a known target protein is similar to any malaria protein, you don’t want to: ❌ BLAST the entire PDB complex
✅ You want to BLAST each chain separately

Reason:
Each chain may correspond to:

    A different protein

    A functional domain

    A protein from another species (in complexes)

If you BLAST the whole thing → you get garbage results.
If you BLAST each chain → you will correctly detect homologs in malaria proteins (taxonomy 5833).


#🎯 What BLAST Can Return

When you run BLASTp against PDB or a filtered NCBI database with taxID = 5833, each result hit will contain:

    Subject sequence ID (sseqid) → This is usually the PDB ID + Chain or a UniProt ID

    Alignment score, E-value, percent identity → Already useful

    Sometimes, the definition line of the matched sequence → Contains the actual protein name

You can customize the output format to include:

✅ sseqid → PDB ID or UniProt ID

✅ stitle → Full title of matched protein

✅ taxid → Organism tax ID

✅ slen → Length of matched sequence

✅ evalue, pident, bitscore → Scores

What we want:

Matched PDB/UniProt ID

Protein name

Any metadata


In [ ]:
def extract_protein_chains(pdb_file_path):
    """
    Extracts protein chain sequences from a PDB file and saves them as FASTA files.

    Args:
        pdb_file_path (str): Path to the PDB file.

    Returns:
        list: List of generated FASTA file paths.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("structure", pdb_file_path)
    ppb = PPBuilder()
    chain_fastas = []

    pdb_path = Path(pdb_file_path)
    output_dir = pdb_path.parent
    pdb_name = pdb_path.stem

    for model in structure:
        for chain in model:
            peptides = ppb.build_peptides(chain)
            if peptides:
                sequence = ''.join(str(peptide.get_sequence()) for peptide in peptides)
                if sequence:
                    chain_id = chain.id
                    fasta_path = output_dir / f"{pdb_name}_{chain_id}.fasta"
                    with open(fasta_path, "w") as f:
                        f.write(f">{pdb_name}_{chain_id}\n{sequence}\n")
                    chain_fastas.append(str(fasta_path))
    return chain_fastas

folders_path = '/content/drive/MyDrive/Drug Repurposing Project/PubChem_PDB_Results'

# Iterate through each folder in folders_path
for folder_name in os.listdir(folders_path):
    folder_path = os.path.join(folders_path, folder_name)
    # Check if it's a directory
    if os.path.isdir(folder_path):
        # Iterate through each file in the folder
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.pdb'):
                file_path = os.path.join(folder_path, file_name)
                extract_protein_chains(file_path)
                print(f"Processed {file_name} in {folder_name}")




# **Do the blast search with each .fasta file sequence and toxonomy = 5833**

✅ Step 3: BLAST Against Malaria Proteins

Now you have the sequences of drug target proteins.

Goal:
Find if these human targets are similar to any malaria proteins (Taxonomy ID 5833).

How:
You run BLASTp (protein-protein alignment):

    For each chain FASTA sequence

    Against a filtered BLAST database of only malaria proteins

    Save the results (scores, % identity, alignment, PDB ID of malaria proteins)

If you find similar malaria proteins → there’s a chance the drug may bind to them.
✅ Step 4: Save Top 3 Matches

From the BLAST output, for each target chain:

    Pick the top 3 similar malaria proteins

    Save their details (ID, score, identity, etc.) in a JSON file

    If no hit found, mark as "NAN"

In [ ]:
def blast_sequence(fasta_file, tax_id="5833"):

    """Extract sequence from FASTA and perform remote BLAST."""
    with open(fasta_file, 'r') as f:
        sequence = ''.join([line.strip() for line in f if not line.startswith(">")])

    print(f"[INFO] Running BLAST for: {fasta_file}")

    output_json = os.path.splitext(fasta_file)[0] + ".json"


    """ Run the BlastP on the information above from the .fasta files which is the sequence for each chain"""
    try:
        result_handle = NCBIWWW.qblast(
            program="blastp",
            database="pdb",
            sequence=sequence,
            entrez_query=f"txid{tax_id}[ORGN]",
            hitlist_size=3
        )

        blast_record = NCBIXML.read(result_handle)

        results = []

        if not blast_record.alignments:
            print("[INFO] No result found.")
            results.append({"message": "No result found."})
        else:
            print(f"[✓] BLAST Results for {fasta_file}:")
            for alignment in blast_record.alignments:
                hsp = alignment.hsps[0]  # Take the best HSP
                result = {
                    "subject_id": alignment.accession,
                    "title": alignment.title,
                    "percent_identity": hsp.identities / hsp.align_length * 100,
                    "alignment_length": hsp.align_length,
                    "evalue": hsp.expect,
                    "bitscore": hsp.score
                }
                results.append(result)
                print(f"  → Subject ID: {result['subject_id']}")
                print(f"     Title    : {result['title']}")
                print(f"     Identity : {hsp.identities}/{hsp.align_length}")
                print(f"     E-value  : {hsp.expect}")
                print(f"     Score    : {hsp.score}\n")


        """ Save results to JSON -> whatever happens we will save a json file for each of the .fasta files whether we have results or no results """
        with open(output_json, "w") as f:
            json.dump(results, f, indent=4)
        print(f"[✓] Results saved to: {output_json}")

    except Exception as e:
        print(f"[!] Error during BLAST: {e}")
        with open(output_json, "w") as f:
            json.dump({"error": str(e)}, f, indent=4)
        print(f"[!] Error info saved to: {output_json}")

In [ ]:
"""Usage"""

parent_folder = '/content/drive/MyDrive/Drug Repurposing Project/PubChem_PDB_Results'

for folder in os.listdir(parent_folder):  # Loop over folders
    folder_path = os.path.join(parent_folder, folder)

    if os.path.isdir(folder_path):  # Make sure it's a folder
        for file in os.listdir(folder_path):  # Loop over files inside the folder
            file_path = os.path.join(folder_path, file)

            if file.endswith('.fasta'):  # Only FASTA files
                blast_sequence(file_path)


✅ Step 5: Download Matched Malaria Protein Structures

For each of the top malaria proteins found in Step 4:

    If they have a PDB structure → Download it.

    If not → Download the AlphaFold predicted structure.

Why? You will use this structure later to:

    Analyze binding

    Run docking

    See if the drug can bind to this malaria protein

🚀 Why are we doing this?

The whole logic is:

    Known drug binds to known human target.

    If that target looks like a malaria protein → the drug may also bind to the malaria protein.

    By checking structural similarity, you reduce the search space of possible binding sites.



In [ ]:
# --- 2. Sequence Extraction ---

# --- 3. BLAST Functions ---
def blast_sequence(sequence, tax_id="5833"):
    """Performs a BLAST search against the PDB database."""
    result_handle = NCBIWWW.qblast(
        program="blastp",
        database="pdb",
        sequence=sequence,
        entrez_query=f"txid{tax_id}[ORGN]",
        hitlist_size=3
    )
    return NCBIXML.read(result_handle)

def safe_blast_with_timeout(sequence, timeout=600):
    """Performs BLAST with a timeout to prevent indefinite waiting."""
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(blast_sequence, sequence)
        try:
            return future.result(timeout=timeout)
        except concurrent.futures.TimeoutError:
            print("⏰ Timeout reached! Skipping this chain.")
            return None

# --- 4. UniProt PDB Mapping ---
def get_pdb_for_accession(accession):
    """Retrieves PDB IDs associated with a UniProt accession."""
    url = f"https://rest.uniprot.org/uniprotkb/{accession}.json"
    response = requests.get(url)
    if response.ok:
        data = response.json()
        pdbs = []
        for xref in data.get("uniProtKBCrossReferences", []):
            if xref["database"] == "PDB":
                pdbs.append(xref["id"])
        return pdbs
    return []

# --- 5. PDB and AlphaFold Download ---
def download_file(url, dest_path, description):
    """Downloads a file from a URL."""
    if os.path.exists(dest_path):
        print(f"📦 Skipping already downloaded {description}")
        return dest_path
    r = requests.get(url)
    if r.ok:
        with open(dest_path, 'w') as f:
            f.write(r.text)
        print(f"📥 Downloaded {description}")
        return dest_path
    print(f"⚠️ Failed to download {description}")
    return None

def download_pdb(pdb_id, dest_folder):
    """Downloads a PDB file."""
    dest_path = os.path.join(dest_folder, f"{pdb_id}.pdb")
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    return download_file(url, dest_path, f"PDB {pdb_id}")

def download_alphafold(uniprot_id, dest_folder):
    """Downloads an AlphaFold model."""
    file_name = f"AF-{uniprot_id}.pdb"
    dest_path = os.path.join(dest_folder, file_name)
    url = f"https://alphafold.ebi.ac.uk/files/{file_name}"
    return download_file(url, dest_path, f"AlphaFold model for {uniprot_id}")

# --- 6. Main Processing Loop ---
for folder_name in os.listdir(BASE_PATH):
    folder_path = os.path.join(BASE_PATH, folder_name)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.pdb'):
                pdb_file_path = os.path.join(folder_path, file)
                pdb_id = file.replace('.pdb', '')
                pubchem_id = folder_name

                print(f"\n📄 Processing {pdb_id} from PubChem {pubchem_id}")
                protein_chains = extract_protein_chains(pdb_file_path)

                for chain_id, sequence in protein_chains.items():
                    json_name = f"{pubchem_id}_{pdb_id}_{chain_id}.json"
                    json_file_path = os.path.join(SAVE_PATH, json_name)

                    if os.path.exists(json_file_path):
                        print(f"⏭️ Skipping chain {chain_id} (already processed)")
                        continue

                    print(f"\n🧬 Chain {chain_id} | Length: {len(sequence)}")
                    result = safe_blast_with_timeout(sequence)

                    if result and result.alignments:
                        top_hits = []
                        for alignment in result.alignments[:3]:
                            hsp = alignment.hsps[0]
                            pdb_code, chain_code = None, None
                            parts = alignment.hit_id.split('|')
                            if alignment.hit_id.startswith('pdb|') and len(parts) >= 3:
                                pdb_code, chain_code = parts[1], parts[2]

                            hit = {
                                "hit_def": alignment.hit_def,
                                "e_value": hsp.expect,
                                "score": hsp.score,
                                "query": hsp.query[:60],
                                "subject": hsp.sbjct[:60],
                                "pdb_code": pdb_code,
                                "chain": chain_code
                            }

                            acc = parts[1] if parts else None
                            if acc:
                                mapped_pdbs = get_pdb_for_accession(acc)
                                hit['mapped_pdbs'] = mapped_pdbs
                                for pdb_id_hit in mapped_pdbs:
                                    download_pdb(pdb_id_hit, PDB_SAVE_PATH)
                                if not mapped_pdbs:
                                    download_alphafold(acc, PDB_SAVE_PATH)

                            top_hits.append(hit)

                        with open(json_file_path, 'w') as f:
                            json.dump(top_hits, f, indent=2)
                        print(f"💾 Saved results to {json_file_path}")

                    else:
                        print(f"❌ No valid BLAST result for {pdb_id} Chain {chain_id}")
                        no_hits_file = os.path.join(SAVE_PATH, "no_pdb_hits.txt")
                        with open(no_hits_file, 'a') as f:
                            f.write(f"{pubchem_id}_{pdb_id}_{chain_id}\n")

#Alignment & Visualization

##Goal: For each row in your dataset

  - Align human target and malaria structure (PDB)

  - Generate an image or structural alignment plot

  - Work entirely in Python in Colab

In [ ]:
# Paths
BASE_DIR = "/content/drive/MyDrive/Drug Repurposing Project/PDB_Pipeline"
PDB_DIR = os.path.join(BASE_DIR, "pdbs")
SNAP_DIR = os.path.join(BASE_DIR, "Snapshots")

os.makedirs(PDB_DIR, exist_ok=True)
os.makedirs(SNAP_DIR, exist_ok=True)

#Fetch and align structures using Bio.PDB
def download_pdb(pdb_chain_id):
    pdb_code = pdb_chain_id.split("_")[0].lower()
    pdb_file = os.path.join(PDB_DIR, f"{pdb_code}.pdb")

    if not os.path.exists(pdb_file):
        pdbl = PDBList()
        raw_path = pdbl.retrieve_pdb_file(pdb_code, file_format='pdb', pdir=PDB_DIR)
        # raw_path will be like: /content/pdb_pipeline/pdbs/pdbXXXX.ent
        new_path = os.path.join(PDB_DIR, f"{pdb_code}.pdb")
        os.rename(raw_path, new_path)

    return pdb_file

# Align and get RMSDfrom Bio.PDB import PDBParser, Superimposer

def extract_chain(structure, chain_id):
    for model in structure:
        for chain in model:
            if chain.id == chain_id:
                return chain
    return None

def align_structures(pdb1_id, pdb2_id):
    parser = PDBParser(QUIET=True)
    pdb1_path = download_pdb(pdb1_id)
    pdb2_path = download_pdb(pdb2_id)

    struct1 = parser.get_structure("target", pdb1_path)
    struct2 = parser.get_structure("malaria", pdb2_path)

    chain1_id = pdb1_id.split("_")[1]
    chain2_id = pdb2_id.split("_")[1]

    chain1 = extract_chain(struct1, chain1_id)
    chain2 = extract_chain(struct2, chain2_id)

    if chain1 is None or chain2 is None:
        return None, None, None, None

    atoms1 = [a for a in chain1.get_atoms() if a.id == 'CA']
    atoms2 = [a for a in chain2.get_atoms() if a.id == 'CA']

    min_len = min(len(atoms1), len(atoms2))
    atoms1 = atoms1[:min_len]
    atoms2 = atoms2[:min_len]

    si = Superimposer()
    si.set_atoms(atoms1, atoms2)
    si.apply(chain2.get_atoms())

    aligned_res1 = [a.get_parent().get_id()[1] for a in atoms1]
    aligned_res2 = [a.get_parent().get_id()[1] for a in atoms2]

    return struct1, struct2, si.rms, (aligned_res1, aligned_res2)

# Store results
results = []

def save_aligned_structure(structure, output_path):

    io = PDBIO()
    io.set_structure(structure)
    io.save(output_path)

    csv_path = "/content/drive/MyDrive/Drug Repurposing Project/target_malaria_dataset.csv"
    df = pd.read_csv(csv_path)

    for idx, row in df.iterrows():
        try:
            target_id = row['target_chain_id']        # e.g., "2X9E_A"
            malaria_id = row['malaria_match_id']      # e.g., "5DYK_A"

            struct1, struct2, rmsd, aligned_res = align_structures(target_id, malaria_id)

            if aligned_res is not None and len(aligned_res[0]) > 0:
                aligned_pdb_name = f"{target_id}_aligned_with_{malaria_id}.pdb"
                aligned_pdb_path = os.path.join(PDB_DIR, aligned_pdb_name)

                save_aligned_structure(struct2, aligned_pdb_path)

                results.append({
                    "target": target_id,
                    "malaria": malaria_id,
                    "rmsd": rmsd,
                    "aligned_pdb": aligned_pdb_name
                })


                print(f"[{idx}] Saved aligned PDB: {aligned_pdb_name} | RMSD: {rmsd:.2f}")
            else:
                print(f"[{idx}] Skipped: Could not align chains or no aligned residues")
        except Exception as e:
            print(f"[{idx}] Error for {row['target_chain_id']} vs {row['malaria_match_id']}: {e}")



In [ ]:
# Convert to DataFrame
rmsd_df = pd.DataFrame(results)

# Filter by RMSD threshold
threshold = 5.0
filtered_df = rmsd_df[rmsd_df['rmsd'] < threshold]

# Save to CSV
rmsd_df.to_csv(os.path.join(BASE_DIR, "/content/drive/MyDrive/Drug Repurposing Project/PDB_Pipeline/all_rmsd_results.csv"), index=False)
filtered_df.to_csv(os.path.join(BASE_DIR, "/content/drive/MyDrive/Drug Repurposing Project/PDB_Pipeline/filtered_rmsd_below_5.csv"), index=False)

# Plot histogram
plt.figure(figsize=(8, 5))
plt.hist(rmsd_df['rmsd'], bins=30, color='skyblue', edgecolor='black')
plt.axvline(threshold, color='red', linestyle='--', label=f'RMSD < {threshold}')
plt.xlabel("RMSD")
plt.ylabel("Frequency")
plt.title("Distribution of Structural Alignment RMSDs")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df_rmsd = pd.read_csv("/content/drive/MyDrive/Drug Repurposing Project/PDB_Pipeline/filtered_rmsd_below_5.csv")
filtered = df_rmsd[df_rmsd['rmsd'] < 5.0]
aligned_files_under_5_rmsd = filtered['aligned_pdb'].tolist()

print(aligned_files_under_5_rmsd)

def render_alignment(pdb1_id, pdb2_id, aligned_residues, out_path_jpg):
    try:
        pdb1_path = download_pdb(pdb1_id)
        pdb2_path = download_pdb(pdb2_id)

        with open(pdb1_path) as f1, open(pdb2_path) as f2:
            pdb1_text = f1.read().strip()
            pdb2_text = f2.read().strip()

        if not pdb1_text or not pdb2_text or 'ATOM' not in pdb1_text or 'ATOM' not in pdb2_text:
            print(f"[WARN] Skipping invalid or empty PDB: {pdb1_id} or {pdb2_id}")
            return

        view = py3Dmol.view(width=800, height=600)

        # Load target (model 0) - background gray
        view.addModel(pdb1_text, 'pdb')
        view.setStyle({'model': 0}, {'cartoon': {'color': 'lightgray'}})

        # Highlight aligned residues on target in green
        for res in aligned_residues[0]:
            view.setStyle({'model': 0, 'resi': str(res)}, {'cartoon': {'color': 'green'}, 'stick': {}})

        # Load malaria (model 1) - background light blue
        view.addModel(pdb2_text, 'pdb')
        view.setStyle({'model': 1}, {'cartoon': {'color': 'skyblue'}})

        # Highlight aligned residues on malaria in green
        for res in aligned_residues[1]:
            view.setStyle({'model': 1, 'resi': str(res)}, {'cartoon': {'color': 'green'}, 'stick': {}})

        view.zoomTo()

        # Force render before .png()
        display(view)
        time.sleep(1.5)

        png_data = view.png()
        if png_data is None:
            print(f"[WARN] view.png() returned None for {pdb1_id} vs {pdb2_id}")
            return

        temp_png_path = out_path_jpg.replace(".jpg", ".png")
        with open(temp_png_path, "wb") as f:
            f.write(png_data)

        with Image.open(temp_png_path) as im:
            rgb_im = im.convert('RGB')
            rgb_im.save(out_path_jpg, 'JPEG')

        os.remove(temp_png_path)

    except Exception as outer:
        print(f"[ERROR] Failed render for {pdb1_id} vs {pdb2_id}: {outer}")

#Example to see:
render_alignment("7DI7_A", "8WXW_A", ([1,2,3,4,5], [1,2,3,4,5]), "/content/pdb_pipeline/Snapshots/test_7DI7_A_aligned_with_8WXW_A.png")
render_alignment("1CET_A","1CEQ_A",([1,2,3,4,5], [1,2,3,4,5]),"/content/pdb_pipeline/Snapshots/test_1CET_A_aligned_with_1CEQ_A.pdb")
render_alignment("1J3K_A","1J3I_A",([1,2,3,4,5], [1,2,3,4,5]),"/content/pdb_pipeline/Snapshots/test_1J3K_A_aligned_with_1J3I_A.pdb")

Given:

    aligned_pdb = "1J3K_A_aligned_with_1J3I_A.pdb" ← malaria homolog aligned onto target

    target_pdb = "1J3K.pdb" ← original full target with ligand bound

It:

    Extracts ligand atoms

    Finds residues within 5Å of the ligand (i.e., binding site)

    Loads both structures into py3Dmol

    Colors:

        Target (gray)

        Ligand (yellow)

        Aligned malaria chain (green)

        Overlap between ligand site and aligned malaria → red

In [ ]:
from Bio.PDB import PDBParser, NeighborSearch
import py3Dmol

def get_binding_residues(pdb_file, ligand_resname=None, cutoff=5.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("target", pdb_file)
    atoms = list(structure.get_atoms())
    ns = NeighborSearch(atoms)

    # Auto-detect ligand if not given
    lig_atoms = [a for a in atoms if a.get_parent().id[0] == 'H' and a.element != 'H']
    if ligand_resname:
        lig_atoms = [a for a in lig_atoms if a.get_parent().get_resname() == ligand_resname]

    binding_res = set()
    for lig_atom in lig_atoms:
        close_atoms = ns.search(lig_atom.coord, cutoff)
        for a in close_atoms:
            parent = a.get_parent()
            if parent.id[0] == ' ' and a.get_parent().get_resname() != lig_atom.get_parent().get_resname():
                binding_res.add(parent.id[1])
    return sorted(binding_res)

def visualize_alignment_with_ligand(target_pdb_path, aligned_pdb_path, binding_residues):
    with open(target_pdb_path) as f1, open(aligned_pdb_path) as f2:
        target_pdb = f1.read()
        aligned_pdb = f2.read()

    view = py3Dmol.view(width=900, height=600)

    # Load target PDB (model 0)
    view.addModel(target_pdb, 'pdb')
    view.setStyle({'model': 0}, {'cartoon': {'color': 'lightgray'}})

    # Color ligand (model 0)
    view.setStyle({'model': 0, 'hetflag': True}, {'stick': {'colorscheme': 'yellowCarbon'}})

    # Highlight binding residues on target
    for resi in binding_residues:
        view.setStyle({'model': 0, 'resi': str(resi)}, {'cartoon': {'color': 'red'}, 'stick': {}})

    # Load aligned malaria structure (model 1)
    view.addModel(aligned_pdb, 'pdb')
    view.setStyle({'model': 1}, {'cartoon': {'color': 'green'}, 'stick': {}})

    view.zoomTo()
    view.show()

# Example usage:
aligned_pdb_path = "/content/drive/MyDrive/Drug Repurposing Project/PDB_Pipeline/pdbs/1J3K_A_aligned_with_1J3I_A.pdb"
!wget https://files.rcsb.org/download/1J3K.pdb

target_pdb_path = "/content/1J3K.pdb"

binding_res = get_binding_residues(target_pdb_path, cutoff=5.0)
visualize_alignment_with_ligand(target_pdb_path, aligned_pdb_path, binding_res)

# Example usage:
!wget https://files.rcsb.org/download/1J3K.pdb

# Set file paths
aligned_pdb_path = "/content/drive/MyDrive/Drug Repurposing Project/PDB_Pipeline/pdbs/1J3K_A_aligned_with_1J3I_A.pdb" # aligned malaria chain
target_pdb_path = "/content/1J3K.pdb" # original drug-bound protein

# Visualize alignment vs ligand-binding site
binding_res = get_binding_residues(target_pdb_path, cutoff=5.0)
visualize_alignment_with_ligand(target_pdb_path, aligned_pdb_path, binding_res)
# Example usage:
!wget https://files.rcsb.org/download/7DI7.pdb

# Set file paths
aligned_pdb_path = "/content/drive/MyDrive/Drug Repurposing Project/PDB_Pipeline/pdbs/7DI7_A_aligned_with_8WXW_A.pdb"  # aligned malaria chain
target_pdb_path = "/content/7DI7.pdb"    # original drug-bound protein

# Visualize alignment vs ligand-binding site
binding_res = get_binding_residues(target_pdb_path, cutoff=5.0)
visualize_alignment_with_ligand(target_pdb_path, aligned_pdb_path, binding_res)



## ✅ Conclusion So Far

- We are working with **`target_malaria_dataset.csv`**, which contains BLAST results mapping human target chains to similar malaria protein chains.
- In this notebook, we analyzed that dataset.
- We downloaded all required PDB files for the targets and malaria proteins.
- We then aligned each target chain to its corresponding malaria chain and computed **RMSD** to evaluate structural similarity.
- After alignment, we saved the aligned structures and filtered the best matches (RMSD < 5) into **`filtered_rmsd_below_5.csv`**.
- We visualized these top alignments using `py3Dmol` to better understand the overlap.
- Finally, we aligned the **malaria chain** not just to the individual target chain, but to the **full target PDB**, to see its spatial relation to the original structure.

---

### 🔜 Next Steps

- Analyze how and where the **drug ligand** in the full target PDB interacts.
- Compare this drug-binding region with the **aligned malaria structure** to see if there's meaningful structural overlap.
- This will help us assess whether the drug could potentially bind to the malaria protein — the key goal of this project.